In [ ]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
print(token)
print(os.environ["OPENAI_API_KEY"])
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)


In [ ]:
response.choices[0].message.content

### Chat API : LangChain

In [ ]:
from langchain.memory import ConversationBufferMemory

# Example: Clearing the memory
memory = ConversationBufferMemory()
memory.clear()


In [4]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI( model=model_name)
chat

In [6]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [7]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [ ]:
prompt_template.messages[0].prompt

In [ ]:
prompt_template.messages[0].prompt.input_variables

In [10]:
customer_style = """American English \
in a calm and respectful tone
"""

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [12]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

In [ ]:
print(type(customer_messages))
print(type(customer_messages[0]))

In [14]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [ ]:
print(customer_response.content)

In [16]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [17]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

print(service_messages[0].content)

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

### Output parsers

In [ ]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

In [21]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

In [ ]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=model_name)
response = chat(messages)
print(response.content)

In [ ]:
type(response.content)

### Parse LLM input string into a Python dictionary

In [25]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [26]:
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown."
)

delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1."
)

price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list."
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [28]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [30]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [32]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""


In [33]:
prompt = ChatPromptTemplate.from_template(review_template_2)
messages = prompt.format_messages(text=customer_review,format_instructions=format_instructions)

In [ ]:
print(messages[0].content)

In [35]:
response = chat(messages)

In [ ]:
print(response.content)

In [37]:
output_dict = output_parser.parse(response.content)

In [ ]:
output_dict

In [ ]:
type(output_dict)

In [ ]:
output_dict.get('delivery_days')